In [ ]:
!pip install numpy requests pillow usearch uform

In [ ]:
import sqlite3
import random
from io import BytesIO

import numpy as np
import requests
import usearch
import uform
from PIL import Image

In [52]:
coordinates_ny = (40.7128, -74.0060)  # New York
coordinates_sf = (37.7749, -122.4194)  # San Francisco
coordinates_la = (34.0522, -118.2437)  # Los Angeles
coordinates_ldn = (51.5074, -0.1278)  # London
coordinates_tk = (35.6895, 139.6917)  # Tokyo
coordinates_all = [
    coordinates_ny,
    coordinates_sf,
    coordinates_la,
    coordinates_ldn,
    coordinates_tk,
]

image_urls = [
    "https://images.pexels.com/photos/8561771/pexels-photo-8561771.jpeg?cs=srgb&dl=pexels-abdulwahab-alawadhi-8561771.jpg&fm=jpg",
    "https://techcrunch.com/wp-content/uploads/2013/02/india_bangalore_bus_.jpg",
    "https://www.carscoops.com/wp-content/uploads/2022/04/Driven-Fiat-500-RED-1-1024x576.jpg",
    "https://images.pexels.com/photos/9331953/pexels-photo-9331953.jpeg",
    "https://i.ytimg.com/vi/0SZ6K3BQAAg/maxresdefault.jpg",
    "https://i.redd.it/59qkkf4p5iv51.jpg",
    "https://di-uploads-pod35.dealerinspire.com/kellynissanoflynnfield/uploads/2022/05/2022-Nissan-Rogue-Blue.jpg",
]


In [ ]:
num_entries = 10000
model = uform.get_model("unum-cloud/uform-vl-english")

In [53]:
# Download the images, load them into Pillow, and encode with UForm
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
image_contents = [requests.get(url, headers=headers, allow_redirects=True).content for url in image_urls]
image_objects = [Image.open(BytesIO(x)) for x in image_contents]
image_objects

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=4032x2268>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2560x1920>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x576>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=3895x2344>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1280x720>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=631x920>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1319x877>]

In [55]:
image_embeddings = model.preprocess_image(image_objects)
image_embeddings = model.encode_image(image_embeddings)
image_embeddings.shape

torch.Size([7, 256])

In [ ]:
conn = sqlite3.connect(":memory:")
conn.enable_load_extension(True)
conn.load_extension(usearch.sqlite)


# Create a table that contains: latitude, longitude, license_plate, image_url, image_embedding_f32.
# All of those are random generated, except for embeddings, which match the URL.
cursor = conn.cursor()
cursor.execute(
    f"""
    CREATE TABLE IF NOT EXISTS vector_table (
        id INTEGER PRIMARY KEY,
        latitude FLOAT,
        longitude FLOAT,
        license_plate TEXT,
        image_url TEXT,
        image_embedding_f32 BLOB
    )
"""
)

In [59]:
# Generate and insert random entries
for i in range(num_entries):
    # Generate a random 256-dimensional vector
    image_idx: int = np.random.randint(0, len(image_urls) - 1)
    image_url: str = image_urls[image_idx]
    image_embedding: np.ndarray = image_embeddings[image_idx]

    # The license plate will have "XXX0000" format
    license_plate = "".join(
        [random.choice("ABCDEFGHIJKLMNOPQRSTUVWXYZ") for i in range(3)]
        + [random.choice("0123456789") for i in range(4)]
    )

    # Pick any city
    lat, lon = random.choice(coordinates_all)
    lat += random.uniform(-0.01, 0.01)
    lon += random.uniform(-0.01, 0.01)

    # Insert the vector into the database as scalars
    cursor.execute(
        f"""
        INSERT INTO vector_table (latitude, longitude, license_plate, image_url, image_embedding_f32) VALUES (?, ?, ?, ?, ?)
    """,
        (
            [
                lat,
                lon,
                license_plate,
                image_url,
                image_embedding.detach().numpy().astype(np.float32).tobytes(),
            ]
        ),
    )

# Commit changes
conn.commit()

In [61]:
text_embedding = model.encode_text(model.preprocess_text(["Red Pickup"])).detach().numpy().astype(np.float32)
text_embedding.shape

(1, 256)

In [ ]:
# We then will filter points:
#   - within specific `distance_haversine_meters` radius from New York,
#   - with at most 2 mistakes in the recognized `license_plate`,
#   - sorted by the semantic similarity to the query "red pickup".

In [66]:
ny_lat, ny_lon = coordinates_ny
max_location_distance = 10000  # Example: 10 kilometers
max_plate_distance = 6

# Text embedding for the query "red pickup"
text_embedding = model.encode_text(model.preprocess_text(["Red Pickup"])).detach().numpy().astype(np.float32)

# Query to select entries based on the conditions
query = """
SELECT 
    id,
    latitude,
    longitude,
    license_plate,
    image_url,
    distance_haversine_meters(latitude, longitude, ?, ?) AS location_distance,
    distance_levenshtein(license_plate, 'XXX0000') AS plate_distance,
    distance_cosine_f32(image_embedding_f32, ?) AS semantic_distance
FROM 
    vector_table
WHERE 
    location_distance < ? AND
    plate_distance < ?
ORDER BY 
    semantic_distance ASC, 
    plate_distance ASC;
"""

cursor.execute(query, [ny_lat, ny_lon, text_embedding.tobytes(), max_location_distance, max_plate_distance])

# Fetch the results
results = cursor.fetchall()

# Process the results as needed
for row in results:
    print(row)  # Each row will contain the id, lat, lon, license_plate, image_url, and distances


(5, 40.71923840589961, -73.99628685669865, 'BWN3400', 'https://images.pexels.com/photos/8561771/pexels-photo-8561771.jpeg?cs=srgb&dl=pexels-abdulwahab-alawadhi-8561771.jpg&fm=jpg', 1087.51708984375, 5, 0.5990111827850342)
(82, 40.70742393952692, -74.00781552033413, 'GUS0904', 'https://techcrunch.com/wp-content/uploads/2013/02/india_bangalore_bus_.jpg', 617.0670776367188, 5, 0.6020922660827637)
(154, 40.72229202662463, -73.99961336422612, 'KRQ0660', 'https://techcrunch.com/wp-content/uploads/2013/02/india_bangalore_bus_.jpg', 1184.791259765625, 5, 0.6020922660827637)
(243, 40.719649004490115, -74.01170799133224, 'UCC0035', 'https://techcrunch.com/wp-content/uploads/2013/02/india_bangalore_bus_.jpg', 900.7923583984375, 5, 0.6020922660827637)
(11, 40.71692294865666, -74.0146965402483, 'IXV0116', 'https://i.ytimg.com/vi/0SZ6K3BQAAg/maxresdefault.jpg', 864.529296875, 5, 0.6025271415710449)
(91, 40.70372449088882, -74.01471153435676, 'NWJ6040', 'https://i.ytimg.com/vi/0SZ6K3BQAAg/maxresdefau